In [3]:
import re
import numpy as np
import math
from matplotlib import pyplot as plt
from collections import defaultdict
from enum import Enum

In [4]:
with open("example.txt") as f:
    data = f.read()
    lines = data.strip().split("\n")
    records = data.strip().split("\n\n")

In [71]:
with open("input.txt") as f:
    data = f.read()
    lines = data.strip().split("\n")   
    records = data.strip().split("\n\n")

In [72]:
#board = np.array([list(x) for x in lines])
#plt.matshow(screen)

In [73]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [74]:
ores = Enum("ores",["ore","clay","obsidian","geode"])
ores["ore"]

<ores.ore: 1>

In [75]:
print(lines)

['Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 14 clay. Each geode robot costs 3 ore and 16 obsidian.', 'Blueprint 2: Each ore robot costs 3 ore. Each clay robot costs 3 ore. Each obsidian robot costs 2 ore and 19 clay. Each geode robot costs 2 ore and 12 obsidian.', 'Blueprint 3: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 9 clay. Each geode robot costs 4 ore and 16 obsidian.', 'Blueprint 4: Each ore robot costs 4 ore. Each clay robot costs 3 ore. Each obsidian robot costs 4 ore and 20 clay. Each geode robot costs 2 ore and 15 obsidian.', 'Blueprint 5: Each ore robot costs 3 ore. Each clay robot costs 4 ore. Each obsidian robot costs 4 ore and 18 clay. Each geode robot costs 2 ore and 11 obsidian.', 'Blueprint 6: Each ore robot costs 4 ore. Each clay robot costs 4 ore. Each obsidian robot costs 3 ore and 11 clay. Each geode robot costs 3 ore and 8 obsidian.', 'Blueprint 7: Eac

In [76]:
blueprints = {}
for line in lines:
    num = int(re.search(r"(\d+):",line)[1])
    robots = re.findall(r"Each ([^ ]*) robot costs ([^\.]*)\.",line)
    #print(robots)
    blueprint = {}
    for robot in robots:
        ingredients = {"ore":0,"clay":0,"obsidian":0,"geode":0}
        for ingredient in re.findall(r"(\d+) ([^ ]*)",robot[1]):
            ingredients[ingredient[1]] = int(ingredient[0])
        blueprint[robot[0]] = (ingredients["ore"],ingredients["clay"],ingredients["obsidian"],ingredients["geode"])
    blueprints[num] = blueprint
    #blueprints[num] = (blueprint["ore"],blueprint["clay"],blueprint["obsidian"],blueprint["geode"])

In [77]:
print(blueprints)

{1: {'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 14, 0, 0), 'geode': (3, 0, 16, 0)}, 2: {'ore': (3, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 19, 0, 0), 'geode': (2, 0, 12, 0)}, 3: {'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 9, 0, 0), 'geode': (4, 0, 16, 0)}, 4: {'ore': (4, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (4, 20, 0, 0), 'geode': (2, 0, 15, 0)}, 5: {'ore': (3, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 18, 0, 0), 'geode': (2, 0, 11, 0)}, 6: {'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (3, 11, 0, 0), 'geode': (3, 0, 8, 0)}, 7: {'ore': (2, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 17, 0, 0), 'geode': (3, 0, 19, 0)}, 8: {'ore': (4, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 7, 0, 0), 'geode': (3, 0, 8, 0)}, 9: {'ore': (3, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (3, 15, 0, 0), 'geode': (4, 0, 16, 0)}, 10: {'ore': (3, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 7, 0, 0), 'geode': (2, 0, 9, 0)}, 11: {'ore': (4

In [78]:
((np.array((1,0,0,0)) - np.array((1,0,0,0))) > 0).any()

False

In [114]:
nameToTuple = {"ore":(1,0,0,0),"clay":(0,1,0,0),"obsidian":(0,0,1,0),"geode":(0,0,0,1)}
part2 = True

def canAfford(ores,plan):
    for x,y in zip(ores,plan):
        if x < y:
            return False
    return True
    #return ((np.array(ores) - np.array(plan)) >= 0).all()

def returnOpts(blueprint,state,states):
    global nameToTuple
    states[((0,0,0,0),(0,0,0,0))] = 1

    robots,resources = state
    if not (np.array(resources) > 0).any():
        return states
    #else:
    #    states[(robots,resources)] = 1

    for robot,cost in blueprint.items():
        if canAfford(resources,cost):
            #tmp_robots = np.array(robots) + np.array(nameToTuple[robot])
            #tmp_resources = np.array(resources) - np.array(cost)
            tmp_state = (tuple(nameToTuple[robot]),tuple(-np.array(cost)))
            states[tmp_state] = 1
            #returnOpts(blueprint,tmp_state,states)
    return states

def sort_key(x):
    return x[0][3]*1000 + x[1][3]*100 + sum(x*(i+1)**4 for i,x in enumerate(x[0]))*3 + sum(x*(i+1)**3 for i,x in enumerate(x[1]))

def tryBluePrint(blueprint):
    ores = Enum("ores",["ore","clay","obsidian","geode"])

    goal = "geode"
    
    
    max_cost = np.array([0]*4)
    for _,b in blueprint.items():
        #print(b)
        max_cost = np.maximum(max_cost,np.array(b))
    max_cost[3] = 100
        
        
    # we start with an ore robot. what luck!
    #robots = defaultdict(int)
    robots = (1,0,0,0)
    resources = (0,0,0,0)
    state = (robots,resources)
    states = {state:1}
    minutes = 16
    minutes = 24
    if part2:
        minutes = 32
    for i in range(minutes):
        print("minute ",i)
        new_states = []
        for state in states:
            robots,resources = [np.array(x) for x in state]
            # build stuff
            changes = {}
            returnOpts(blueprint,state,changes)
        
            # collect resources
            resources += robots

            # add new robots
            for change in changes:
                d_robots,d_resources = [np.array(x) for x in change]
                tmp_state = np.array([robots+d_robots,resources+d_resources])
                new_states.append(tmp_state)
        
        print("len new_states are ",len(new_states))
        #for state in new_states:
        #    print(state)
        # prune:
        max_robot_cost = [0]*4
        
        next_states_dict = {}
        max_geode_bot = 0
        sorted_new_states = sorted(new_states,key=sort_key)
       # print("### sorted new states ###",sorted_new_states[-20:])
        for x in sorted_new_states[-1000:]:
            addState = True
            # if max_geode_bot-1 > x[0][3]:
            #     add_state = False
            # elif max_geode_bot < x[0][3]:
            #     max_geode_bot = x[0][3]
            #     print("increased geode_bot max to",max_geode_bot)
            #     #print(x)
            #     #next_states_dict = {}
                
            # adding more robots than the highest cost is just silly
            if (x[0] > max_cost).any():
                add_state = False
            #for y in new_states:
        # This line kills performance! Need a better way
                #if (x <= y).all() and id(x) != id(y):
                    #addState = False
                    #break
            if addState:
                bots = tuple(x[0])
                # there's a better way to do this, but I'm brain dead right now
                if bots not in next_states_dict:
                    next_states_dict[bots] = [x[1]]
                elif bots in next_states_dict:
                    next_states_dict[bots].append(x[1])
                        
                    

        # update
        states = {}
        for k,v in next_states_dict.items():
            for r1 in v:
                addIt = True
                for r2 in v:
                    if id(r1) == id(r2):
                        continue
                    tmp = True
                    for v1,v2 in zip(r1,r2):
                        tmp &= v1 <= v2
                    addIt &= not tmp
                    #if (r1 <= r2).all():
                    #    addIt = False
                if addIt:
                    states[(tuple(k),tuple(r1))] = 1
        #states = {(tuple(k),tuple(v)):1 for k,v in next_states_dict.items()}
        print("len pruned states are",len(states))
        count = 5
        for state in states:
            #print(np.array(state))
            count -= 1
            if count < 0:
                break
        #print(i)
    #print("max obsidian is ",max(x[1][3] for x in states))
    return max(x[1][3] for x in states)

In [115]:
max_cost = np.array([0]*4)
for _,b in blueprint.items():
    print(b)
    max_cost = np.maximum(max_cost,np.array(b))
print(max_cost)

(3, 0, 0, 0)
(4, 0, 0, 0)
(3, 18, 0, 0)
(4, 0, 19, 0)
[ 4 18 19  0]


In [116]:
filtered = {}
for index,blueprint in blueprints.items():
    addIt = True
    for index1,blueprint2 in blueprints.items():
        if id(blueprint) == id(blueprint2):
            continue
        if (np.array([x for _,x in blueprint.items()]) <= np.array([x for _,x in blueprint2.items()])).all():
            addIt = False
    if addIt:
        filtered[index] = blueprint
    print(blueprint)
#%prun tryBluePrint(blueprint)
print(len(blueprints))
print(len(filtered))
print(filtered)

{'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 14, 0, 0), 'geode': (3, 0, 16, 0)}
{'ore': (3, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 19, 0, 0), 'geode': (2, 0, 12, 0)}
{'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 9, 0, 0), 'geode': (4, 0, 16, 0)}
{'ore': (4, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (4, 20, 0, 0), 'geode': (2, 0, 15, 0)}
{'ore': (3, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 18, 0, 0), 'geode': (2, 0, 11, 0)}
{'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (3, 11, 0, 0), 'geode': (3, 0, 8, 0)}
{'ore': (2, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 17, 0, 0), 'geode': (3, 0, 19, 0)}
{'ore': (4, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 7, 0, 0), 'geode': (3, 0, 8, 0)}
{'ore': (3, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (3, 15, 0, 0), 'geode': (4, 0, 16, 0)}
{'ore': (3, 0, 0, 0), 'clay': (3, 0, 0, 0), 'obsidian': (2, 7, 0, 0), 'geode': (2, 0, 9, 0)}
{'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': 

In [120]:
max_v = 0
count = 3
vals = []
for index,blueprint in blueprints.items():
    if count < 0:
        break
    print(blueprint)
    v = tryBluePrint(blueprint)
    vals.append(v)
    max_v += v*index
    print(index,v)
    count -= 1
#%prun tryBluePrint(blueprint)
print("quality value is",max_v)

{'ore': (4, 0, 0, 0), 'clay': (4, 0, 0, 0), 'obsidian': (4, 14, 0, 0), 'geode': (3, 0, 16, 0)}
minute  0
len new_states are  1
len pruned states are 1
minute  1
len new_states are  1
len pruned states are 1
minute  2
len new_states are  1
len pruned states are 1
minute  3
len new_states are  1
len pruned states are 1
minute  4
len new_states are  3
len pruned states are 3
minute  5
len new_states are  5
len pruned states are 3
minute  6
len new_states are  5
len pruned states are 3
minute  7
len new_states are  7
len pruned states are 5
minute  8
len new_states are  11
len pruned states are 7
minute  9
len new_states are  17
len pruned states are 11
minute  10
len new_states are  25
len pruned states are 15
minute  11
len new_states are  43
len pruned states are 29
minute  12
len new_states are  75
len pruned states are 43
minute  13
len new_states are  125
len pruned states are 68
minute  14
len new_states are  195
len pruned states are 103
minute  15
len new_states are  310
len prune

In [124]:
print(vals[0]*vals[1]*vals[2])

4114


In [39]:
foo = np.array(((0,1,0,1),(13,2,14,1)))
print(foo)
bar = tuple(tuple(x) for x in foo)
biz = ((20,1,0,0),(13,2,1,0))

print(bar)
print(sort_key(bar))
sort_key(biz)
sorted([20,1000,10])

[[ 0  1  0  1]
 [13  2 14  1]]
((0, 1, 0, 1), (13, 2, 14, 1))
1134


[10, 20, 1000]

In [ ]:
for index,blueprint in blueprints.items():
    print(blueprint)
%prun tryBluePrint(blueprint)

In [119]:
print(max_v)
# got 1418
# when I lowered the limit to -500, I got 1410
# when raised to -2500, I still get 1418
# and 1418 is correct

14145


In [99]:
v = tryBluePrint(blueprint)
print(v)

minute  0
len new_states are  1
### sorted new states ### [array([[1, 0, 0, 0],
       [1, 0, 0, 0]])]
len pruned states are 1
[[1 0 0 0]
 [1 0 0 0]]
minute  1
len new_states are  1
### sorted new states ### [array([[1, 0, 0, 0],
       [2, 0, 0, 0]])]
len pruned states are 1
[[1 0 0 0]
 [2 0 0 0]]
minute  2
len new_states are  1
### sorted new states ### [array([[1, 0, 0, 0],
       [3, 0, 0, 0]])]
len pruned states are 1
[[1 0 0 0]
 [3 0 0 0]]
minute  3
len new_states are  2
### sorted new states ### [array([[1, 0, 0, 0],
       [4, 0, 0, 0]]), array([[2, 0, 0, 0],
       [1, 0, 0, 0]])]
len pruned states are 2
[[1 0 0 0]
 [4 0 0 0]]
[[2 0 0 0]
 [1 0 0 0]]
minute  4
len new_states are  4
### sorted new states ### [array([[1, 0, 0, 0],
       [5, 0, 0, 0]]), array([[2, 0, 0, 0],
       [2, 0, 0, 0]]), array([[2, 0, 0, 0],
       [3, 0, 0, 0]]), array([[1, 1, 0, 0],
       [1, 0, 0, 0]])]
len pruned states are 3
[[1 0 0 0]
 [5 0 0 0]]
[[2 0 0 0]
 [3 0 0 0]]
[[1 1 0 0]
 [1 0 0 0]]
minut

In [ ]:
# guessed 1235 for part 1, too low
#1361, too low